<a href="https://colab.research.google.com/github/takeshitayy/ipynb2markdown/blob/main/ipynb2markdown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ipynb2markdown

web上に公開されている Jupyter Notebook ファイル (ipynb) を markdown に変換する

google colaboratory で実行する

1. `url` に公開されている Notebook ファイル (ipynb) の URL を設定
1. `ctrl + F9` ですべてを実行する
1. マークダウン記法とプレビューが表示される



In [20]:
import os

#@title ipynb ファイルのURLを指定する
url = "https://raw.githubusercontent.com/openai/openai-cookbook/main/examples/Classification_using_embeddings.ipynb" #@param {type:"string"}

if url.startswith("https://github.com"):
    # url が https://github.com の場合は、ipynb ファイルのurlに変換する
    url = url.replace("https://github.com", "https://raw.githubusercontent.com").replace("/blob", "")

ipynbFilename = os.path.basename(url)
markdownFilename = ipynbFilename + '.md'
print('URL: ' + url)
print('ipynbFilename: ' + ipynbFilename)
print('markdownFilename: '+ markdownFilename)

URL: https://raw.githubusercontent.com/openai/openai-cookbook/main/examples/Classification_using_embeddings.ipynb
ipynbFilename: Classification_using_embeddings.ipynb
markdownFilename: Classification_using_embeddings.ipynb.md


## ipynb ファイルをダウンロード

In [16]:
import requests
response = requests.get(url)

if response.status_code == 200:
    # レスポンスのステータスコードが200（成功）の場合、ファイルを保存
    with open(ipynbFilename, "wb") as file:
        file.write(response.content)

## ipynb ファイルを マークダウンに変換

In [17]:
import os
import nbconvert
from nbconvert import MarkdownExporter

# Markdownエクスポーターを作成
exporter = MarkdownExporter()

 # .ipynbファイルを読み込み
with open(ipynbFilename, "r", encoding="utf-8") as ipynb_file:
    notebook_content = ipynb_file.read()

# .ipynbファイルを.mdファイルに変換
markdownContent, _ = exporter.from_file(ipynbFilename)

print(markdownContent)

## Classification using embeddings

There are many ways to classify text. This notebook shares an example of text classification using embeddings. For many text classification tasks, we've seen fine-tuned models do better than embeddings. See an example of fine-tuned models for classification in [Fine-tuned_classification.ipynb](Fine-tuned_classification.ipynb). We also recommend having more examples than embedding dimensions, which we don't quite achieve here.

In this text classification task, we predict the score of a food review (1 to 5) based on the embedding of the review's text. We split the dataset into a training and a testing set for all the following tasks, so we can realistically evaluate performance on unseen data. The dataset is created in the [Obtain_dataset Notebook](Obtain_dataset.ipynb).



```python
# imports
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.me

## マークダウンの内容を表示

In [18]:
from IPython.display import Markdown

Markdown(markdownContent)

## Classification using embeddings

There are many ways to classify text. This notebook shares an example of text classification using embeddings. For many text classification tasks, we've seen fine-tuned models do better than embeddings. See an example of fine-tuned models for classification in [Fine-tuned_classification.ipynb](Fine-tuned_classification.ipynb). We also recommend having more examples than embedding dimensions, which we don't quite achieve here.

In this text classification task, we predict the score of a food review (1 to 5) based on the embedding of the review's text. We split the dataset into a training and a testing set for all the following tasks, so we can realistically evaluate performance on unseen data. The dataset is created in the [Obtain_dataset Notebook](Obtain_dataset.ipynb).



```python
# imports
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# load data
datafile_path = "data/fine_food_reviews_with_embeddings_1k.csv"

df = pd.read_csv(datafile_path)
df["embedding"] = df.embedding.apply(eval).apply(np.array)  # convert string to array

# split data into train and test
X_train, X_test, y_train, y_test = train_test_split(
    list(df.embedding.values), df.Score, test_size=0.2, random_state=42
)

# train random forest classifier
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
probas = clf.predict_proba(X_test)

report = classification_report(y_test, preds)
print(report)

```

                  precision    recall  f1-score   support
    
               1       0.88      0.35      0.50        20
               2       1.00      0.38      0.55         8
               3       1.00      0.18      0.31        11
               4       1.00      0.26      0.41        27
               5       0.74      1.00      0.85       134
    
        accuracy                           0.77       200
       macro avg       0.92      0.43      0.52       200
    weighted avg       0.82      0.77      0.72       200
    


We can see that the model has learnt to distinguish between the categories decently. 5-star reviews show the best performance overall, and this is not too surprising, since they are the most common in the dataset.


```python
from openai.embeddings_utils import plot_multiclass_precision_recall

plot_multiclass_precision_recall(probas, y_test, [1, 2, 3, 4, 5], clf)

```

    RandomForestClassifier() - Average precision score over all classes: 0.87



    
![png](output_3_1.png)
    


Unsurprisingly 5-star and 1-star reviews seem to be easier to predict. Perhaps with more data, the nuances between 2-4 stars could be better predicted, but there's also probably more subjectivity in how people use the inbetween scores.
